In [ ]:
# Add root folder to ptyhon path

import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [1]:
import numpy as np
import scipy.stats as ss
from matplotlib import pyplot as plt

In [2]:
# Set up the failure mode class
from pof.failure_mode import FailureMode

fm = FailureMode(alpha=50, beta=1.5, gamma=10)

ModuleNotFoundError: No module named 'pof'